In [1]:
import numpy as np
import cv2
import glob
import time
import pickle
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
# from sklearn.cross_validation import train_test_split  # for scikit-learn version <= 0.17
from sklearn.model_selection import train_test_split  # if you are using scikit-learn >= 0.18
from scipy.ndimage.measurements import label

## 1 - Feature extraction functions

In [2]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img,
                                  orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img,
                       orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

    
# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))


# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    
    # Return the individual histograms, bin_centers and feature vector
    return hist_features


# Change image color space
def change_colorspace(img, color_space = 'RGB'):
    if color_space != 'RGB':
        if color_space == 'HSV':
            conv_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            conv_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            conv_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            conv_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            conv_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else:
        conv_image = np.copy(img)
    
    return conv_image


# Extract features from a single image
def extract_features_from_image(img,
                                color_space='RGB',
                                spatial_size=(32, 32),
                                hist_bins=32,
                                orient=9, 
                                pix_per_cell=8,
                                cell_per_block=2,
                                hog_channel='ALL',
                                spatial_feat=True,
                                hist_feat=True,
                                hog_feat=True):
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.array(hog_features)
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)
        
    #print("Feature space size:")    
    #print(spatial_features.shape)
    #print(hist_features.shape)
    #print(hog_features.shape)
        
    #9) Return concatenated array of features
    return np.concatenate(img_features)


# Extract features from a list of images
def extract_features_from_image_list(img_file_list,
                                     color_space='RGB',
                                     spatial_size=(32, 32),
                                     hist_bins=32,
                                     orient=9,
                                     pix_per_cell=8,
                                     cell_per_block=2,
                                     hog_channel='ALL',
                                     spatial_feat=True,
                                     hist_feat=True,
                                     hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    
    # Iterate through the list of images
    for img_file in img_file_list:      
        # Read in each one by one
        img = mpimg.imread(img_file)
        # Extract features from each of them
        img_features = extract_features_from_image(img,
                                                   color_space=color_space,
                                                   spatial_size=spatial_size,
                                                   hist_bins=hist_bins,
                                                   orient=orient, 
                                                   pix_per_cell=pix_per_cell,
                                                   cell_per_block=cell_per_block,
                                                   hog_channel=hog_channel,
                                                   spatial_feat=spatial_feat,
                                                   hist_feat=hist_feat,
                                                   hog_feat=hog_feat)
        
        features.append(img_features)
        
    # Return list of feature vectors
    return features

## 2 - Object detection functions

In [3]:
"""
# Takes one full camera image and define a grid of search windows.
# There's also the option to scale the image (x/y_start_stop still refer to the original size image)
def prepare_windows(img,
                    x_start_stop=[None, None],
                    y_start_stop=[None, None], 
                    xy_window=(64, 64),
                    xy_overlap=(0.5, 0.5)):
    
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step)
    
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
            
    # Return the list of windows
    return window_list
    
    
# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_in_windows(img, 
                      windows,
                      clf, 
                      scaler, 
                      color_space='RGB', 
                      spatial_size=(32, 32),
                      hist_bins=32, 
                      hist_range=(0, 256),
                      orient=9, 
                      pix_per_cell=8,
                      cell_per_block=2, 
                      hog_channel=0,
                      spatial_feat=True, 
                      hist_feat=True,
                      hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    
    #2) Iterate over all windows in the list
    for window in windows:
        
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))
        
        #4) Extract features for that window using extract_features_from_image()
        features = extract_features_from_image(test_img,
                                               color_space=color_space, 
                                               spatial_size=spatial_size,
                                               hist_bins=hist_bins, 
                                               orient=orient,
                                               pix_per_cell=pix_per_cell, 
                                               cell_per_block=cell_per_block, 
                                               hog_channel=hog_channel,
                                               spatial_feat=spatial_feat, 
                                               hist_feat=hist_feat,
                                               hog_feat=hog_feat)
        
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
            
    #8) Return windows for positive detections
    return on_windows
"""

# Search the image for cars.
# It uses image-wide hog and creates its own windows.
def search_in_image_with_fixed_window_size(img,
                                           y_start_stop,
                                           scale,
                                           classifier,
                                           scaler,
                                           color_space,
                                           hog_channel='ALL',      # Image channels on which we want to compute HOG
                                           hog_orient=9,           # HOG possible orientations
                                           hog_pix_per_cell=8,     # HOG number of px per cell
                                           hog_cell_per_block=2,   # HOG number of cells per block
                                           spatial_size=(32,32),   # SPATIAL features resize
                                           histogram_bins=32,      # HISTOGRAM number of bins
                                           enable_spatial_features=True,
                                           enable_hist_features=True,
                                           enable_hog_features=True):
    
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    
    img_tosearch = img[y_start_stop[0]:y_start_stop[1],:,:]
    ctrans_tosearch = change_colorspace(img_tosearch, color_space=color_space)
    
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    # Separate channels
    if hog_channel == 'ALL':
        ch1 = ctrans_tosearch[:,:,0]
        ch2 = ctrans_tosearch[:,:,1]
        ch3 = ctrans_tosearch[:,:,2]
    else:
        ch1 = ctrans_tosearch[:,:,hog_channel]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // hog_pix_per_cell) - 1 # - hog_cell_per_block
    nyblocks = (ch1.shape[0] // hog_pix_per_cell) - 1 # - hog_cell_per_block
    nfeat_per_block = hog_orient * hog_cell_per_block ** 2
    
    # Compute HOG features over the whole image
    if hog_channel == 'ALL':
        hog1 = get_hog_features(ch1, hog_orient, hog_pix_per_cell, hog_cell_per_block, feature_vec=False)
        hog2 = get_hog_features(ch2, hog_orient, hog_pix_per_cell, hog_cell_per_block, feature_vec=False)
        hog3 = get_hog_features(ch3, hog_orient, hog_pix_per_cell, hog_cell_per_block, feature_vec=False)
    else:
        hog1 = get_hog_features(ch1, hog_orient, hog_pix_per_cell, hog_cell_per_block, feature_vec=False)
        
    # 64 was the original sampling rate, with 8 cells and 8 pix per cell
    window_size = 64
    nblocks_per_window = (window_size // hog_pix_per_cell) -1 # + 1  - hog_cell_per_block
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Create an empty list to receive positive detection windows
    on_windows = []
    
    # Iterate over all windows in the list
    for xb in range(nxsteps):
        for yb in range(nysteps):
            
            ypos = yb * cells_per_step
            xpos = xb * cells_per_step
            
            # Extract HOG for this patch
            if hog_channel == 'ALL':
                hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
                hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
                hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
                hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))
            else:
                hog_features = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()

            xleft = xpos * hog_pix_per_cell
            ytop = ypos * hog_pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window_size, xleft:xleft+window_size], (64,64))
          
            # Spatial features
            if enable_spatial_features:
                spatial_features = bin_spatial(subimg, size=spatial_size)
            else:
                spatial_features = []
                
            #print(spatial_features.shape)
                
            # Histogram features
            if enable_hist_features:
                hist_features = color_hist(subimg, nbins=histogram_bins)
            else:
                hist_features = []
                
            #print(hist_features.shape)
            #print(hog_features.shape)
            
            # Stack all features
            features = np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1)
            #print(features.shape)
            
            # Scale extracted features to be fed to classifier
            scaled_features = scaler.transform(features)
            
            # Predict using your classifier
            prediction = svc.predict(scaled_features)
            
            # If positive (prediction == 1) then save the window
            show_all_windows = 0
            if prediction == 1 or show_all_windows:
                xbox_left = np.int(xleft*scale)
                xbox_right = np.int(xleft*scale) + np.int(window_size*scale)
                ybox_top = np.int(ytop*scale) + y_start_stop[0]
                ybox_bottom = ybox_top + np.int(window_size*scale)
                on_windows.append(((xbox_left, ybox_top),(xbox_right, ybox_bottom)))
    
    return on_windows


def search_in_image(img,
                    search_settings,
                    classifier,
                    scaler,
                    color_space,
                    hog_channel='ALL',
                    hog_orient=9,
                    hog_pix_per_cell=8,
                    hog_cell_per_block=2,
                    spatial_size=(32,32),
                    histogram_bins=32,
                    enable_spatial_features=True,
                    enable_hist_features=True,
                    enable_hog_features=True):
    
    on_windows = []
    
    for setting in search_settings:
        new_on_windows = search_in_image_with_fixed_window_size(img,
                                                                y_start_stop=setting[0],
                                                                scale=setting[1],
                                                                classifier=classifier,
                                                                scaler=scaler,
                                                                color_space=color_space,
                                                                hog_channel=hog_channel,
                                                                hog_orient=hog_orient,
                                                                hog_pix_per_cell=hog_pix_per_cell,
                                                                hog_cell_per_block=hog_cell_per_block,
                                                                spatial_size=spatial_size,
                                                                histogram_bins=histogram_bins,
                                                                enable_spatial_features=enable_spatial_features,
                                                                enable_hist_features=enable_hist_features,
                                                                enable_hog_features=enable_hog_features)
        
        # print("{} detections with setting {}".format(len(new_on_windows), setting))
        on_windows.extend(new_on_windows)
    
    return on_windows
        

# Draw bounding boxes
def draw_boxes(img, bboxes, color=(0,0,255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy


def add_heat(heatmap, bbox_list):
    for box in bbox_list:
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    return heatmap
    
    
def apply_threshold(heatmap, threshold):
    heatmap[heatmap < threshold] = 0
    return heatmap


def draw_labeled_bboxes(img, labels):
    # Empty list containing the bounding boxes for each label
    bbox_list = []
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y and append it to the list
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        bbox_list.append(bbox)
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img, bbox_list


class Vehicles():
    def __init__(self):
        self.fifo = [] 
        
    def add_windows(self, windows):
        self.fifo.append(windows)
        if len(self.fifo) > 15:
            self.fifo = self.fifo[len(self.fifo)-15:]

## Classifier

In [ ]:
cars = glob.glob('classification_datasets/vehicles/**/*.png', recursive=True)
notcars = glob.glob('classification_datasets/non-vehicles/**/*.png', recursive=True)
print('Car images: {}'.format(len(cars)))
print('Non-car images: {}'.format(len(notcars)))

# sample_size = 1000
sample_size = None
if sample_size is not None:
    cars = cars[0:sample_size]
    notcars = notcars[0:sample_size]

## Feature parameters (MUST BE THE SAME FOR BOTH THE CLASSIFICATION AND DETECTION ON IMAGES)
color_space = 'YCrCb'             # Can be RGB, HSV, LUV, HLS, YUV, YCrCb

hog_channel = 'ALL'             # Can be 0, 1, 2, or "ALL"
hog_orient = 11                 # HOG possible orientations
hog_pix_per_cell = 16           # HOG number of px per cell
hog_cell_per_block = 2          # HOG number of cells per block

spatial_size = (32, 32)         # Spatial binning dimensions

histogram_bins = 16             # Number of histogram bins

enable_spatial_features = True  # Spatial features on or off
enable_hist_features = True     # Histogram features on or off
enable_hog_features = True      # HOG features on or off

# Load images and extract features
t0=time.time()
car_features = extract_features_from_image_list(cars,
                                                color_space=color_space, 
                                                spatial_size=spatial_size,
                                                hist_bins=histogram_bins, 
                                                orient=hog_orient,
                                                pix_per_cell=hog_pix_per_cell, 
                                                cell_per_block=hog_cell_per_block, 
                                                hog_channel=hog_channel,
                                                spatial_feat=enable_spatial_features, 
                                                hist_feat=enable_hist_features,
                                                hog_feat=enable_hog_features)
notcar_features = extract_features_from_image_list(notcars,
                                                   color_space=color_space, 
                                                   spatial_size=spatial_size,
                                                   hist_bins=histogram_bins, 
                                                   orient=hog_orient,
                                                   pix_per_cell=hog_pix_per_cell, 
                                                   cell_per_block=hog_cell_per_block, 
                                                   hog_channel=hog_channel,
                                                   spatial_feat=enable_spatial_features, 
                                                   hist_feat=enable_hist_features,
                                                   hog_feat=enable_hog_features)
t_extract = time.time()


# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)
print(X.shape)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y,
                                                    test_size=0.2,
                                                    random_state=rand_state)


# Define and fit model
print('Feature vector length:', len(X_train[0]))
svc = LinearSVC()
svc.fit(X_train, y_train)
t_fit = time.time()


# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
t_accuracy = time.time()

model = {"model": svc,
         "scaler": X_scaler}
pickle.dump(model, open("model.p", "wb"))


print()
print('Time stats')
print('Time to load images and extract features: {}'.format(t_extract-t0))
print('Time to fit model: {}'.format(t_fit-t_extract))
print('Time to compute model accuracy: {}'.format(t_accuracy-t_fit))

## Pipeline

In [ ]:
# Where to search
search_settings = (((400, 400+48), 0.75),
                   ((400, 400+64), 1.0),
                   ((408, 408+64), 1.0),
                   ((416, 416+64), 1.0),
                   ((400, 400+96), 1.5),
                   ((432, 432+96), 1.5),
                   ((400, 400+128), 2.0),
                   ((432, 432+128), 2.0),
                   ((400, 400+192), 3.0),
                   ((464, 660+192), 3.0))

def pipeline(image):
    
    debug = 1
    
    if debug:
        draw_image = np.copy(image)
    
    # Uncomment the following line if you extracted training
    # data from .png images (scaled 0 to 1 by mpimg) and the
    # image you are searching is a .jpg (scaled 0 to 255)
    image = image.astype(np.float32)/255
    
    # Search for cars in the image
    on_windows = search_in_image(image,
                                 search_settings,
                                 classifier=svc,
                                 scaler=scaler,
                                 color_space=color_space,
                                 hog_channel=hog_channel,
                                 hog_orient=hog_orient,
                                 hog_pix_per_cell=hog_pix_per_cell,
                                 hog_cell_per_block=hog_cell_per_block,
                                 spatial_size=spatial_size,
                                 histogram_bins=histogram_bins,
                                 enable_spatial_features=enable_spatial_features,
                                 enable_hist_features=enable_hist_features,
                                 enable_hog_features=enable_hog_features)
    
    if debug:
        on_windows_img = draw_boxes(draw_image, on_windows)
        
    # New empty heatmap
    heatmap = np.zeros_like(image[:,:,0]).astype(np.float)
    
    # Add the detected
    if len(on_windows) > 0:
        detections.add_windows(on_windows)
        
    # Populate the heatmap based on these and historic data
    for item in detections.fifo:
        heatmap = add_heat(heatmap, item)
    
    # Apply threshold to help remove false positives
    heatmap = cv2.GaussianBlur(heatmap,(101,101),0)
    heatmap = apply_threshold(heatmap,1)
    heatmap = np.clip(heatmap, 0, 255)
    labels = label(heatmap)
    labeled_img, bbox_list = draw_labeled_bboxes(draw_image, labels)
    
    if debug:
        f, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(20,10))
        ax1.imshow(on_windows_img)
        ax2.imshow(heatmap, cmap='hot')
        ax3.imshow(labeled_img, cmap='gray')
    
    return labeled_img

## Detect cars in camera image

In [ ]:
# Load classifier and scaler
model = pickle.load(open("model.p", "rb"))
svc = model["model"]
scaler = model["scaler"]

# Read camera image
test_files = glob.glob('test_images/*.jpg')
# test_files = ['test_images/test1.jpg']


for file in test_files:
    image = mpimg.imread(file)
    detections = Vehicles()
    
    labelled_img = pipeline(image)
    
# https://github.com/jeremy-shannon/CarND-Vehicle-Detection/blob/master/vehicle_detection_project.ipynb


## Detect cars in video

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip

# filename = "test_video.mp4"
filename = "project_video.mp4"
clip = VideoFileClip(filename)
white_output = filename[:-4] + '_output.mp4'

# Video boundaries
start_second = 4
end_second = 6

detections = Vehicles()
white_clip = clip.fl_image(pipeline)#.subclip(start_second,end_second)

%time white_clip.write_videofile(white_output, audio=False)